In [2]:
import json
from pathlib import Path
import sys
sys.path.append('..')
import paths

In [3]:
#ground truth file artpedia2wiki_combined.json
path_gt_annotations=paths.COMBINED_ANNOTATIONS_PATH
path_falcon_annotations=Path("falcon_res.json")

In [4]:
gt_annotations=json.load(open(path_gt_annotations))
falcon_annotations=json.load(open(path_falcon_annotations))

In [5]:
#limit the gt to the test set
gt_annotations={k:v for k,v in gt_annotations.items() if v["split"]=="test"}
len(gt_annotations)

100

In [6]:
def find_key(query, keys):
    if query in keys:
        return query
    #find a key that starts with or ends with query
    for key in keys:
        if key.startswith(query) or key.endswith(query):
            return key
    #find the other way around
    for key in keys:
        if query.startswith(key) or query.endswith(key):
            return key
    #find a key that contains query
    for key in keys:
        if query in key:
            return key
    return None


In [7]:
def group_falcon_mentions(falcon_mentions):
    mentions_groups={}
    current_group=[]
    current_key=None
    for mention in falcon_mentions:
        if current_key==mention["surface form"]:
            current_group.append(mention)
        if current_key!=mention["surface form"]:
            if current_key!=None:
                mentions_groups[current_key]=current_group
            current_group=[]
            current_group.append(mention)
            current_key=mention["surface form"]
        if len(current_group)==50:
            mentions_groups[current_key]=current_group
            current_group=[]
    if len(current_group)>0:
        mentions_groups[current_key]=current_group
    return mentions_groups

In [8]:
ranks=[]
el_matches_length=0
for qid,gt_annotation in gt_annotations.items():
    visual_sentences=gt_annotation["visual_sentences"]
    visual_el_matches=gt_annotation["visual_el_matches"]
    contextual_sentences=gt_annotation["contextual_sentences"]
    contextual_el_matches=gt_annotation["contextual_el_matches"]
    #concate the visual and contextual sentences
    all_sentences=visual_sentences+contextual_sentences
    all_el_matches=visual_el_matches+contextual_el_matches
    el_matches_length+=sum(len(el_matches) for el_matches in all_el_matches)
    for sentence,el_matches in zip(all_sentences,all_el_matches):
        sentence_ranks=[]
        #find the sentence in falcon_annotations
        falcon_sentence=None
        for falcon_sentences in falcon_annotations[qid]:
            if falcon_sentences["text"]==sentence:
                falcon_sentence=falcon_sentences
                break
        if falcon_sentence:
            falcon_mentions=falcon_sentence["falcon"]["entities_wikidata"]
            mentions_groups=group_falcon_mentions(falcon_mentions)
            for el_match in el_matches:
                qid_gt=el_match["qid"]
                surface_form_gt=el_match["text"]
                key=find_key(surface_form_gt,mentions_groups.keys())
                if key:
                    falcon_mentions=mentions_groups[key]
                    uri_falcon=[falcon_mention["URI"] for falcon_mention in falcon_mentions]
                    try:
                        rank=uri_falcon.index(qid_gt)+1
                        sentence_ranks.append(rank)
                    except ValueError:
                        sentence_ranks.append(None)
                    # for falcon_mention in falcon_mentions:
                    #     qid_falcon=falcon_mention["URI"]
                    #     if qid_falcon==qid_gt:
                    #         try:
                    #             rank=falcon_mentions.index(falcon_mention)+1
                    #             sentence_ranks.append(rank)
                    #         except ValueError:
                    #             sentence_ranks.append(None)
                else:
                    sentence_ranks.append(None)
        else:
            if el_matches:
                for el_match in el_matches:
                    sentence_ranks.append(None)
                print("Sentence not found in falcon_annotations")
        assert len(sentence_ranks)==len(el_matches)
        ranks.extend(sentence_ranks)
ranks[:10]    

[None, None, None, None, 1, None, None, None, None, None]

In [9]:
len(ranks)

645

In [10]:
el_matches_length

645

In [11]:
k_values=[1,3,5,10,20]
for k in k_values:
    #compute hits@k
    hits_k=sum([1 for rank in ranks if rank and rank<=k])
    print(f"hits@{k}={hits_k/el_matches_length}")

hits@1=0.09147286821705426
hits@3=0.13333333333333333
hits@5=0.15503875968992248
hits@10=0.16279069767441862
hits@20=0.1937984496124031


In [12]:
#number of mentions that are not found in falcon_annotations
sum([1 for rank in ranks if rank==None])

520

In [13]:
sum([1 for rank in ranks if rank==None])/el_matches_length

0.8062015503875969